In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Cell 0 — run once
!pip install jiwer
!pip install -q librosa==0.9.2 jiwer==2.7.0 tqdm
# (evaluate or datasets not required here; we use jiwer for WER/CER)

import os
import random
import json
import glob
import math
import time
from pathlib import Path
from typing import List, Dict

import numpy as np
import pandas as pd
import librosa
import torch
import torchaudio
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from jiwer import wer, cer

# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.5 MB/s eta 0:00:0000:0100:01
ERROR: Could not find a version that satisfies the requirement jiwer==2.7.0 (from versions: 1.2, 1.3, 1.3.1, 1.3.2, 2.0.0, 2.0.1, 2.1.0, 2.2.0, 2.2.1, 2.3.0, 2.4.0, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.0.4, 3.0.5, 3.1.0, 4.0.0)
ERROR: No matching distribution found for jiwer==2.7.0
Device: cuda


In [3]:
# Cell 1
ROOT = Path("/kaggle/input/sanskrit/Audio_Dataset")
print("Dataset root:", ROOT)
print("Exists:", ROOT.exists())
for p in sorted(ROOT.iterdir()):
    print(p.name, "→", len(list(p.iterdir())) if p.is_dir() else "file")
    
# print small listing from each folder
for folder in ["sp002", "sp025", "Transcript"]:
    path = ROOT / folder
    print("\n", folder, "exists:", path.exists())
    if path.exists():
        print(" example:", list(path.glob("*"))[:5])


Dataset root: /kaggle/input/sanskrit/Audio_Dataset
Exists: True
Transcript → 2
sp002 → 2705
sp025 → 923

 sp002 exists: True
 example: [PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-002227_RV_07.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-002444_RV_07.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-002145_RV_05.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-001451_RV_05.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-001347_RV_02.wav')]

 sp025 exists: True
 example: [PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp025/sp025-000731_004.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp025/sp025-000704_004.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp025/sp025-000305_002.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp025/sp025-000304_002.wav'), PosixPath('/kaggle/input/sanskrit/Audio_Dataset/sp025/sp025-000609_004.wav')]

 Transcript exists: True
 example: [Posix

In [4]:
# Cell 2 - create mapping if not present
MAP_CSV = Path("audio_transcript_mapping.csv")
if MAP_CSV.exists():
    df_mapping = pd.read_csv(MAP_CSV)
    print("Loaded mapping:", MAP_CSV, "->", len(df_mapping))
else:
    # read transcripts (one file per speaker with lines)
    trans_dir = ROOT / "Transcript"
    def read_lines(p): 
        return [l.strip() for l in open(p, 'r', encoding='utf-8').read().splitlines() if l.strip()]
    sp2 = read_lines(trans_dir / "sp002.txt")
    sp25 = read_lines(trans_dir / "sp025.txt")
    # gather audio files (sorted)
    sp2_audio = sorted(map(str, (ROOT/"sp002").glob("*.wav")))
    sp25_audio = sorted(map(str, (ROOT/"sp025").glob("*.wav")))
    print("sp002 audio:", len(sp2_audio), "lines:", len(sp2))
    print("sp025 audio:", len(sp25_audio), "lines:", len(sp25))
    # pair by index (warn if counts mismatch)
    pairs=[]
    for i, a in enumerate(sp2_audio):
        if i < len(sp2): pairs.append({"audio_path": a, "transcript": sp2[i], "speaker":"sp002"})
    for i, a in enumerate(sp25_audio):
        if i < len(sp25): pairs.append({"audio_path": a, "transcript": sp25[i], "speaker":"sp025"})
    df_mapping = pd.DataFrame(pairs)
    df_mapping.to_csv(MAP_CSV, index=False)
    print("Saved mapping to", MAP_CSV, "Total pairs:", len(df_mapping))
df_mapping.head()


sp002 audio: 2705 lines: 2705
sp025 audio: 923 lines: 923
Saved mapping to audio_transcript_mapping.csv Total pairs: 3628


,audio_path,transcript,speaker
0,/kaggle/input/sanskrit/Audio_Dataset/sp002/sp0...,sp002-000001_KS_04\tमूर्च्छिता रतिः इत्युक्तम्,sp002
1,/kaggle/input/sanskrit/Audio_Dataset/sp002/sp0...,sp002-000002_KS_04\tसम्प्रति तद्वृत्तान्तमेवाह,sp002
2,/kaggle/input/sanskrit/Audio_Dataset/sp002/sp0...,sp002-000003_KS_04\tअथेति,sp002
3,/kaggle/input/sanskrit/Audio_Dataset/sp002/sp0...,sp002-000004_KS_04\tअथ अनन्तरम् मोहो मूर्च्छा ...,sp002
4,/kaggle/input/sanskrit/Audio_Dataset/sp002/sp0...,sp002-000005_KS_04\tपरायणम् अभिप्रेते तत्परे प...,sp002


In [5]:
# Cell 3
from sklearn.model_selection import train_test_split

df = df_mapping.copy().dropna(subset=["audio_path","transcript"])
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df["speaker"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df["speaker"])
print(len(train_df), len(val_df), len(test_df))
train_df.to_csv("train_split.csv", index=False)
val_df.to_csv("val_split.csv", index=False)
test_df.to_csv("test_split.csv", index=False)


2902 363 363


In [6]:
# Cell 4
VOCAB_FILE = Path("vocab.json")
if VOCAB_FILE.exists():
    with open(VOCAB_FILE, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    print("Loaded vocab.json size", len(vocab))
else:
    # build from transcripts (char-level)
    all_text = " ".join(df["transcript"].astype(str).tolist())
    unique_chars = sorted(set(all_text))
    vocab = {ch: idx for idx, ch in enumerate(unique_chars)}
    # add specials
    vocab.update({"[PAD]": len(vocab), "[UNK]": len(vocab)+1, "[BLANK]": len(vocab)+2})
    with open(VOCAB_FILE, 'w', encoding='utf-8') as f:
        json.dump(vocab, f, ensure_ascii=False, indent=2)
    print("Saved vocab.json size", len(vocab))
# invert for id->char
id2char = {v:k for k,v in vocab.items()}
vocab_size = len(vocab)
print("Vocab size", vocab_size)


Saved vocab.json size 84
Vocab size 84


In [14]:
# === REPLACEMENT CELL for Cell 5: robust torchaudio-based feature extraction + SpecAugment ===
import torchaudio
import torchaudio.functional as Fta
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB, Resample
import numpy as np
import torch

SAMPLE_RATE = 16000
N_MELS = 80
N_FFT = 400         # 25 ms @ 16k
HOP_LENGTH = 160    # 10 ms @ 16k

# Precreate transforms (faster)
mel_spec_transform = MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=N_FFT,
    hop_length=HOP_LENGTH,
    n_mels=N_MELS,
    power=2.0,        # power spectrogram
    normalized=False,
)
db_transform = AmplitudeToDB(stype='power')

def wav_to_mel(path, sr=SAMPLE_RATE, n_mels=N_MELS):
    """
    Load audio with torchaudio, compute Mel (dB) + delta + delta-delta,
    normalize per-utterance and return stacked feature (240, T).
    """
    # load waveform (channels, samples), torchaudio returns float32
    waveform, sr0 = torchaudio.load(path)
    # convert to mono
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    # resample if needed
    if sr0 != sr:
        waveform = Resample(sr0, sr)(waveform)

    # waveform shape: (1, samples)
    # compute mel spectrogram -> (n_mels, time) after squeeze
    mel = mel_spec_transform(waveform)          # (1, n_mels, T)
    mel_db = db_transform(mel)                  # (1, n_mels, T)
    mel_db = mel_db.squeeze(0)                  # (n_mels, T) as Tensor

    # compute deltas using torchaudio.functional.compute_deltas
    delta = Fta.compute_deltas(mel_db)          # (n_mels, T)
    delta2 = Fta.compute_deltas(delta)          # (n_mels, T)

    # convert to numpy and stack: (n_mels*3, T)
    mel_np = mel_db.cpu().numpy()
    delta_np = delta.cpu().numpy()
    delta2_np = delta2.cpu().numpy()
    stacked = np.concatenate([mel_np, delta_np, delta2_np], axis=0)  # (240, T)

    # per-utterance normalization (safe)
    stacked = (stacked - stacked.mean()) / (stacked.std() + 1e-9)
    return stacked.astype(np.float32)  # (240, T)

# SpecAugment (same semantics as before)
def spec_augment(spec, time_mask_param=30, freq_mask_param=13, num_time_masks=1, num_freq_masks=1):
    """
    spec: numpy array (240, T)
    Returns augmented numpy array.
    """
    spec = spec.copy()
    _, T = spec.shape
    # time masks
    for _ in range(num_time_masks):
        t = np.random.randint(0, time_mask_param + 1)
        if t <= 0 or T - t <= 0:
            continue
        t0 = np.random.randint(0, T - t)
        spec[:, t0:t0+t] = 0
    # freq masks applied only to mel bands rows 0:80
    for _ in range(num_freq_masks):
        f = np.random.randint(0, freq_mask_param + 1)
        if f <= 0 or 80 - f <= 0:
            continue
        f0 = np.random.randint(0, 80 - f)
        spec[f0:f0+f, :] = 0
    return spec


In [15]:
# Cell 6
def text_to_ids(text, vocab_map):
    ids = []
    for ch in text:
        ids.append(vocab_map.get(ch, vocab_map.get("[UNK]")))
    return ids

class SanskritSpectrogramDataset(Dataset):
    def __init__(self, df, vocab_map, augment=False):
        self.df = df.reset_index(drop=True)
        self.vocab = vocab_map
        self.augment = augment
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        spec = wav_to_mel(row['audio_path'])   # (240, T)
        if self.augment:
            spec = spec_augment(spec)
        # ATS: transpose to (T, feat)
        spec_t = spec.T   # (T, feat=240)
        label_ids = text_to_ids(row['transcript'], self.vocab)
        sample = {"inputs": spec_t, "labels": np.array(label_ids, dtype=np.int64), "audio_path": row['audio_path']}
        return sample


In [16]:
# Cell 7
def collate_fn(batch):
    # batch: list of {"inputs": (T, feat), "labels": (L,), ...}
    batch_inputs = [torch.tensor(b["inputs"], dtype=torch.float32) for b in batch]
    lengths = torch.tensor([t.shape[0] for t in batch_inputs], dtype=torch.long)
    # pad inputs on time dimension
    max_len = max([t.shape[0] for t in batch_inputs])
    feat_dim = batch_inputs[0].shape[1]
    padded = torch.zeros((len(batch_inputs), max_len, feat_dim), dtype=torch.float32)
    for i, t in enumerate(batch_inputs):
        padded[i, :t.shape[0], :] = t
    # labels padded with pad id
    label_list = [torch.tensor(b["labels"], dtype=torch.long) for b in batch]
    max_lab = max([l.numel() for l in label_list])
    pad_id = vocab["[PAD]"]
    label_padded = torch.full((len(label_list), max_lab), fill_value=pad_id, dtype=torch.long)
    label_lens = torch.tensor([l.numel() for l in label_list], dtype=torch.long)
    for i, l in enumerate(label_list):
        label_padded[i, :l.numel()] = l
    return {
        "inputs": padded,             # (B, T, feat)
        "input_lengths": lengths,     # (B,)
        "labels": label_padded,       # (B, L)
        "label_lengths": label_lens
    }


In [21]:
# Cell 8
train_dataset = SanskritSpectrogramDataset(train_df, vocab, augment=True)
val_dataset   = SanskritSpectrogramDataset(val_df, vocab, augment=False)
test_dataset  = SanskritSpectrogramDataset(test_df, vocab, augment=False)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)

print("train/val/test batches:", len(train_loader), len(val_loader), len(test_loader))


train/val/test batches: 726 91 91


In [30]:
# ============================================
# 🧠 HYBRID ASR MODEL — BiLSTM + Attention + CTC
# FP16 safe, AMP compatible, ready for Kaggle
# ============================================

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random

# ---------------- Encoder ---------------- #
class EncoderBiLSTM(nn.Module):
    def __init__(self, input_dim=240, cnn_channels=128, lstm_hidden=512, num_layers=2, dropout=0.3):
        super().__init__()
        # Convolutional frontend for local feature extraction
        self.conv = nn.Sequential(
            nn.Conv1d(input_dim, cnn_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(cnn_channels)
        )
        # Bidirectional LSTM encoder
        self.lstm = nn.LSTM(
            input_size=cnn_channels,
            hidden_size=lstm_hidden,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout
        )
        self.proj = nn.Linear(2 * lstm_hidden, 2 * lstm_hidden)

    def forward(self, x, lengths):
        # x: (B, T, feat)
        x = x.transpose(1, 2)            # (B, feat, T)
        x = self.conv(x)                 # (B, C, T)
        x = x.transpose(1, 2)            # (B, T, C)

        # pack sequences for LSTM
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out_packed, _ = self.lstm(packed)
        out, out_lengths = pad_packed_sequence(out_packed, batch_first=True)

        out = self.proj(out)  # (B, T', 2*hidden)
        return out, out_lengths


# ---------------- Attention Decoder ---------------- #
class AttentionDecoder(nn.Module):
    def __init__(self, enc_dim, hidden_dim, vocab_size, emb_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=vocab["[PAD]"])
        self.attn = nn.Linear(enc_dim + emb_dim, hidden_dim)
        self.v_attn = nn.Linear(hidden_dim, 1)
        self.lstm = nn.LSTMCell(emb_dim + enc_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward_step(self, prev_token, prev_hidden, encoder_outputs, mask):
        # prev_token: (B,)
        emb = self.embedding(prev_token)  # (B, emb)
        B, T, C = encoder_outputs.size()

        # Attention mechanism
        emb_exp = emb.unsqueeze(1).expand(-1, T, -1)      # (B, T, emb)
        att_in = torch.cat([encoder_outputs, emb_exp], 2) # (B, T, enc+emb)
        energy = torch.tanh(self.attn(att_in))            # (B, T, hidden)
        scores = self.v_attn(energy).squeeze(-1)          # (B, T)

        # ===== FP16-safe masking =====
        if mask.dtype != torch.bool:
            mask = mask.bool()
        neg_inf = torch.finfo(scores.dtype).min  # safe min value for dtype
        scores = scores.masked_fill(~mask, neg_inf)
        # ==============================

        alpha = torch.softmax(scores, dim=1)              # (B, T)
        context = torch.bmm(alpha.unsqueeze(1), encoder_outputs).squeeze(1)  # (B, enc_dim)

        hx, cx = self.lstm(torch.cat([emb, context], dim=1), prev_hidden)
        logits = self.out(hx)  # (B, vocab)
        return logits, (hx, cx), alpha

    def forward(self, targets, encoder_outputs, enc_mask, teacher_forcing_ratio=1.0):
        B, L = targets.size()
        device = targets.device
        outputs = []
        hx = torch.zeros(B, self.lstm.hidden_size, device=device)
        cx = torch.zeros(B, self.lstm.hidden_size, device=device)
        prev = torch.full((B,), vocab["[PAD]"], dtype=torch.long, device=device)

        for t in range(L):
            logits, (hx, cx), alpha = self.forward_step(prev, (hx, cx), encoder_outputs, enc_mask)
            outputs.append(logits.unsqueeze(1))
            # Teacher forcing
            use_teacher = random.random() < teacher_forcing_ratio
            prev = targets[:, t] if use_teacher else logits.argmax(dim=-1)

        outputs = torch.cat(outputs, dim=1)  # (B, L, vocab)
        return outputs


# ---------------- Hybrid Model ---------------- #
class HybridASRModel(nn.Module):
    def __init__(self, vocab_size, enc_input_dim=240):
        super().__init__()
        self.encoder = EncoderBiLSTM(input_dim=enc_input_dim)
        enc_dim = 2 * 512
        self.ctc_fc = nn.Linear(enc_dim, vocab_size)
        self.decoder = AttentionDecoder(enc_dim, hidden_dim=512, vocab_size=vocab_size)

    def forward(self, inputs, input_lengths, labels=None, label_lengths=None, teacher_forcing_ratio=1.0):
        enc_out, enc_lens = self.encoder(inputs, input_lengths)
        ctc_logits = self.ctc_fc(enc_out)  # (B, T, V)

        # Create attention mask safely
        max_time = enc_out.size(1)
        device = inputs.device
        if isinstance(enc_lens, (tuple, list)):
            enc_lens = torch.tensor(enc_lens, device=device)
        else:
            enc_lens = enc_lens.to(device)
        enc_mask = torch.arange(max_time, device=device).unsqueeze(0) < enc_lens.unsqueeze(1)

        attn_logits = None
        if labels is not None:
            attn_logits = self.decoder(labels, enc_out, enc_mask, teacher_forcing_ratio)
        return ctc_logits, attn_logits, enc_lens


In [31]:
# Cell 10
ctc_loss_fn = nn.CTCLoss(blank=vocab["[BLANK]"], zero_infinity=True, reduction='mean')

def compute_losses(ctc_logits, attn_logits, enc_lens, labels, label_lens, alpha=0.4):
    # ctc_logits: (B, T, V) -> for CTCLoss need (T, B, V)
    log_probs = torch.log_softmax(ctc_logits, dim=-1)  # (B, T, V)
    log_probs_TBV = log_probs.permute(1,0,2)           # (T, B, V)
    input_lengths = enc_lens
    target_lengths = label_lens
    # flatten labels by removing pad (assumes pad id at end)
    # CTC expects targets as 1D concatenated
    targets = []
    for i in range(labels.size(0)):
        targets.append(labels[i, :label_lens[i]].cpu().numpy().tolist())
    targets_concat = torch.tensor([t for seq in targets for t in seq], dtype=torch.long).to(log_probs.device)
    # build targets offset: pass as CPU 1D? PyTorch CTC allows 1D targets
    # But we will call with list approach: requires concat and lengths
    # Use ctc_loss_fn
    # PyTorch's CTCLoss requires targets as 1D tensor
    ctc_loss = ctc_loss_fn(log_probs_TBV, targets_concat, input_lengths, target_lengths)
    # Attention CE loss
    attn_loss = torch.tensor(0.0, device=log_probs.device)
    if attn_logits is not None:
        # attn_logits: (B, L, V), labels: (B, L)
        B, L, V = attn_logits.size()
        attn_logits_flat = attn_logits.view(B*L, V)
        labels_flat = labels.view(B*L)
        # CE ignoring pad id
        attn_loss = nn.functional.cross_entropy(attn_logits_flat, labels_flat, ignore_index=vocab["[PAD]"])
    total = alpha * ctc_loss + (1.0 - alpha) * attn_loss
    return total, ctc_loss.detach(), attn_loss.detach()


In [32]:
# === DEBUG: quick single-sample test (run after replacing wav_to_mel) ===
from pathlib import Path
# pick one audio path from df_mapping (ensure df_mapping exists)
sample_path = df_mapping.iloc[0]['audio_path'] if 'df_mapping' in globals() else None
print("Sample audio:", sample_path)
if sample_path is not None:
    feat = wav_to_mel(sample_path)
    print("Feature shape (240, T):", feat.shape)
    # try augment
    aug = spec_augment(feat)
    print("Augmented shape:", aug.shape)
else:
    print("df_mapping not found in memory — re-run mapping cell first.")


Sample audio: /kaggle/input/sanskrit/Audio_Dataset/sp002/sp002-000001_KS_04.wav
Feature shape (240, T): (240, 306)
Augmented shape: (240, 306)


In [33]:
# Cell 11
model = HybridASRModel(vocab_size).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

NUM_EPOCHS = 10
ALPHA = 0.4   # weight for CTC
best_wer = 1.0
save_dir = Path("./hybrid_checkpoints")
save_dir.mkdir(exist_ok=True)

for epoch in range(NUM_EPOCHS):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False)
    running_loss = 0.0
    for batch in pbar:
        inputs = batch["inputs"].to(DEVICE)        # (B, T, feat)
        in_lens = batch["input_lengths"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        lab_lens = batch["label_lengths"].to(DEVICE)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            ctc_logits, attn_logits, enc_lens = model(inputs, in_lens, labels, lab_lens, teacher_forcing_ratio=0.9)
            loss, ctc_l, attn_l = compute_losses(ctc_logits, attn_logits, enc_lens, labels, lab_lens, alpha=ALPHA)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        pbar.set_postfix({"loss": f"{running_loss / (pbar.n + 1):.4f}"})

    # --- validation at epoch end (you can also evaluate every N steps)
    model.eval()
    all_preds = []
    all_refs = []
    with torch.no_grad():
        for vb in tqdm(val_loader, desc="Val", leave=False):
            v_inputs = vb["inputs"].to(DEVICE)
            v_in_lens = vb["input_lengths"].to(DEVICE)
            v_labels = vb["labels"].to(DEVICE)
            v_label_lens = vb["label_lengths"].to(DEVICE)
            # forward without teacher forcing (greedy decode)
            ctc_logits, attn_logits, enc_lens = model(v_inputs, v_in_lens, labels=None, label_lengths=None, teacher_forcing_ratio=0.0)
            # decode CTC greedy on ctc_logits
            probs = torch.softmax(ctc_logits, dim=-1)  # (B,T,V)
            preds = probs.argmax(dim=-1).cpu().numpy()  # (B,T)
            # collapse repeats & remove blank
            for i, p in enumerate(preds):
                prev = None
                out = []
                for tok in p:
                    if tok == vocab["[BLANK]"]: 
                        prev = tok; continue
                    if tok == prev: 
                        prev = tok; continue
                    out.append(id2char.get(int(tok), ""))
                    prev = tok
                pred_str = "".join(out)
                # ground truth
                lab = v_labels[i, :v_label_lens[i]].cpu().numpy()
                ref_str = "".join([id2char.get(int(x), "") for x in lab])
                all_preds.append(pred_str)
                all_refs.append(ref_str)
    val_wer = wer(all_refs, all_preds)
    val_cer = cer(all_refs, all_preds)
    print(f"Epoch {epoch+1} — AvgLoss: {running_loss/len(train_loader):.4f}  Val WER: {val_wer:.4f}  CER: {val_cer:.4f}")
    # checkpoint if better
    if val_wer < best_wer:
        best_wer = val_wer
        torch.save(model.state_dict(), save_dir / "best_model.pt")
        print("Saved best model (WER improved).")


/tmp/ipykernel_37/1507901773.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


Epoch 1/10:   0%|          | 0/726 [00:00<?, ?it/s]

/tmp/ipykernel_37/1507901773.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


Val:   0%|          | 0/91 [00:00<?, ?it/s]

Epoch 1 — AvgLoss: 2.9604  Val WER: 1.0000  CER: 0.9092


...

Val:   0%|          | 0/91 [00:00<?, ?it/s]

Epoch 9 — AvgLoss: 1.1721  Val WER: 0.7585  CER: 0.1756
Saved best model (WER improved).


Epoch 10/10:   0%|          | 0/726 [00:00<?, ?it/s]

Val:   0%|          | 0/91 [00:00<?, ?it/s]

Epoch 10 — AvgLoss: 1.0692  Val WER: 0.7262  CER: 0.1541
Saved best model (WER improved).


In [35]:
!pip install -q kenlm pyctcdecode

import os, re, glob

# Gather all transcript text
TRANSCRIPT_PATH = "/kaggle/input/sanskrit/Audio_Dataset/Transcript"
texts = []

for txt_file in glob.glob(os.path.join(TRANSCRIPT_PATH, "*.txt")):
    with open(txt_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            line = re.sub(r"\s+", " ", line.strip())
            if len(line) > 0:
                texts.append(line)

print(f"Collected {len(texts)} lines for LM training")

# Save corpus
with open("lm_corpus.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(texts))

# Train kenlm 3-gram model
!lmplz -o 3 < lm_corpus.txt > sanskrit.arpa
!build_binary sanskrit.arpa sanskrit.klm

print("✅ Trained 3-gram Sanskrit LM: sanskrit.klm")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 9.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 26.5 MB/s eta 0:00:00
Collected 3628 lines for LM training
/bin/bash: line 1: lmplz: command not found
/bin/bash: line 1: build_binary: command not found
✅ Trained 3-gram Sanskrit LM: sanskrit.klm


In [38]:
from pyctcdecode import build_ctcdecoder

# Load vocabulary
labels = list(vocab.keys())

# Build beam search decoder WITHOUT LM
decoder = build_ctcdecoder(
    labels=labels,
    kenlm_model_path=None,  # no LM file
    alpha=0.0,  # LM weight (unused)
    beta=0.0   # word insertion penalty
)

print("✅ Beam search decoder (no LM) ready!")


✅ Beam search decoder (no LM) ready!


In [39]:
from jiwer import wer, cer
from tqdm import tqdm
import numpy as np

def evaluate_dataset(dataset, name="Validation"):
    model.eval()
    all_preds, all_refs = [], []

    with torch.no_grad():
        for batch in tqdm(DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn, num_workers=0),
                          desc=f"Evaluating {name}"):
            inputs = batch["inputs"].to(DEVICE)
            in_lens = batch["input_lengths"].to(DEVICE)
            labels = batch["labels"]
            
            # Forward pass
            with torch.amp.autocast("cuda", enabled=(DEVICE=="cuda")):
                ctc_logits, attn_logits, enc_lens = model(inputs, in_lens)

            # Get predictions (CTC probabilities)
            probs = torch.nn.functional.log_softmax(ctc_logits, dim=-1)
            probs_np = probs.cpu().detach().numpy()[0]

            # Decode
            pred_text = decoder.decode(probs_np)
            label_text = ''.join([list(vocab.keys())[i] for i in labels[0].tolist() if i in vocab.values()])

            all_preds.append(pred_text)
            all_refs.append(label_text)

    # Compute metrics
    w = wer(all_refs, all_preds)
    c = cer(all_refs, all_preds)
    print(f"\n📊 {name} — WER: {w:.4f}, CER: {c:.4f}")
    return all_refs, all_preds

# Evaluate on validation & test
val_refs, val_preds = evaluate_dataset(val_dataset, "Validation")
test_refs, test_preds = evaluate_dataset(test_dataset, "Test")


Evaluating Validation: 100%|██████████| 363/363 [02:04<00:00,  2.91it/s]



📊 Validation — WER: 1.1351, CER: 4.4331


Evaluating Test: 100%|██████████| 363/363 [02:16<00:00,  2.67it/s]


📊 Test — WER: 1.1147, CER: 4.4365


In [48]:
import torch
import numpy as np
from jiwer import wer, cer
from tqdm.notebook import tqdm

# Ensure model is in eval mode
model.eval()

# Reverse vocab (id → char)
id2char = {v: k for k, v in vocab.items()}

# --------------- 1️⃣ Greedy CTC decoding ----------------
def greedy_ctc_decode(logits, blank_id):
    """
    logits: (T, vocab_size)
    blank_id: index of [BLANK] in vocab
    Returns: list of token IDs (collapsed, blanks removed)
    """
    pred_ids = np.argmax(logits, axis=-1)
    prev = blank_id
    output = []
    for p in pred_ids:
        if p != prev and p != blank_id:
            output.append(p)
        prev = p
    return output

# --------------- 2️⃣ Evaluation loop ----------------
pred_texts, ref_texts = [], []

print("⚡ Running fast greedy decoding on test set...")

for batch in tqdm(test_loader, total=len(test_loader)):
    inputs = batch["inputs"].to(DEVICE)
    input_lengths = batch["input_lengths"].to(DEVICE)
    labels = batch["labels"].to(DEVICE)
    label_lengths = batch["label_lengths"].to(DEVICE)

    with torch.no_grad(), torch.amp.autocast("cuda", enabled=(DEVICE == "cuda")):
        ctc_logits, attn_logits, enc_lens = model(inputs, input_lengths)
        probs = torch.nn.functional.log_softmax(ctc_logits, dim=-1)

    # Convert to numpy
    probs_np = probs[0].cpu().numpy()

    # Handle blank token safely
    if "[BLANK]" in vocab:
        blank_id = vocab["[BLANK]"]
    elif "[CTC_BLANK]" in vocab:
        blank_id = vocab["[CTC_BLANK]"]
    else:
        blank_id = 0  # fallback for safety

    # Decode predictions
    token_ids = greedy_ctc_decode(probs_np, blank_id)
    pred_text = "".join([id2char[i] for i in token_ids if i in id2char])

    # Decode reference (remove pad/blank/unk)
    label_np = labels[0].cpu().numpy().tolist()
    ref_text = "".join([
        id2char[i] for i in label_np
        if i not in [-100, vocab.get("[PAD]", -1), blank_id, vocab.get("[UNK]", -1)]
    ])

    pred_texts.append(pred_text)
    ref_texts.append(ref_text)

# --------------- 3️⃣ Compute metrics ----------------
final_wer = wer(ref_texts, pred_texts)
final_cer = cer(ref_texts, pred_texts)

print("\n📊 Final Evaluation (Greedy Decode):")
print(f"   ✅ WER: {final_wer:.4f}")
print(f"   ✅ CER: {final_cer:.4f}")

# --------------- 4️⃣ Show sample predictions ----------------
for i in range(5):
    print(f"\n🔹 Sample {i+1}")
    print(f"REF:  {ref_texts[i][:250]}")
    print(f"PRED: {pred_texts[i][:250]}")


⚡ Running fast greedy decoding on test set...


  0%|          | 0/91 [00:00<?, ?it/s]


📊 Final Evaluation (Greedy Decode):
   ✅ WER: 0.6988
   ✅ CER: 0.1514

🔹 Sample 1
REF:  sp025-000259_002	कार्येऽस्मिन् संलग्नेभ्यः सर्वेभ्यः मन्त्रालयेभ्यः सर्वेभ्यः विभागेभ्यः केन्द्रस्य राज्य प्रशासनानाम् च सर्वेभ्यः कर्मकरेभ्यः हृदयेन भूरिशः वर्धापनानि वितरामि
PRED: sp025-0007_002	तार्येस्मिन् सलल्लद्नेभ्याः सर्वेभ्यः मन्त्रालयैभ्यः सर्वेभ्य विभादेभ्यः तेन््रस्य नाज्य प्रशासनानाम् च सर्वेभ् कर्म करेभ्यः विदेन भूरिषः वर्धापनानि वितरामि

🔹 Sample 2
REF:  sp002-001298_RV_02	अरुन्धतीम् च सवत्साम् धेनुम् च प्रदक्षिणीकृत्य प्रगतो दक्षिणम् प्रदक्षिणम्
PRED: sp002-0017_RV_02	अरुन्धतीञ् च सवत्ससाम् देनुञ् च प्रदक्षिणे कृत्य प्रगतो दक्षिणम् प्रदक्षिणम्

🔹 Sample 3
REF:  sp025-000381_002	अस्मदीयाः ये निर्धनाः शिल्पिनः मूर्ति निर्मातारः सन्ति ते आजीविकाम् प्राप्स्यन्ति
PRED: sp025-00032_00	समदीयाायेनिर्धनाशीपिदः मूति निर्मातारः सन्तिते आजीरिकाम् प्रास्यन्ति

🔹 Sample 4
REF:  sp002-001148_RV_02	न तु तव इत्यर्थः
PRED: sp002-0017_RV_02	मतु तव इत्यर्थः

🔹 Sample 5
REF:  sp002-002455_RV_07	सूर्यम् 